In [8]:
# Extract text from pdf 
from PyPDF2 import PdfReader
import io

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

pdf_text_pypdf = extract_text_from_pdf('RC1.pdf')

In [62]:
with open('RC_1000.txt','r+',encoding='charmap', errors='ignore') as f:
    pdf_text = f.read()

In [9]:
import re
import os
import pandas as pd

# Function to read text from file with charmap encoding and ignore errors
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='charmap', errors='ignore') as file:
        text = file.read()
    return text

# Function to extract passages, questions, and choices
def extract_data(text):
    # Regular expression to match the passage
    passage_pattern = r'(?s)(.*?)1\.\s'
    passage_match = re.search(passage_pattern, text)

    # Extract passage
    passage = passage_match.group(1).strip() if passage_match else ""

    # Regular expression to match questions and choices
    questions_pattern = r'(\d+\.\s.*?)(?=\d+\.\s|\Z)'
    questions_matches = re.findall(questions_pattern, text, re.DOTALL)

    questions = []
    choices = []

    for question_match in questions_matches:
        # Extract question
        question_pattern = r'\d+\.\s(.*?)(?=\(A\)\s|$)'
        question_text = re.search(question_pattern, question_match, re.DOTALL).group(1).strip()
        questions.append(question_text)

        # Extract choices
        choices_pattern = r'\([A-E]\)\s(.*?)(?=\([A-E]\)\s|$)'
        choices_texts = re.findall(choices_pattern, question_match, re.DOTALL)
        choices.append({f"Choice {str(i)}": choice_text.strip() for i, choice_text in enumerate(choices_texts)})

    return passage, questions, choices

# Main function to process all files in the "passages" folder
def process_all_files(folder_path, output_csv_path):
    data = {
        "Passage": [],
        "Question": [],
        "Choices": []
    }
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            text = read_text_from_file(file_path)
            passage, questions, choices = extract_data(text)
            data["Passage"].extend([passage] * len(questions))
            data["Question"].extend(questions)
            data["Choices"].extend(choices)
    
    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)

In [11]:

# Usage
folder_path = 'passages'
output_csv_path = 'output.csv'
process_all_files(folder_path, output_csv_path)

# Print confirmation
print("Processed all files and saved to", output_csv_path)

Processed all files and saved to output.csv
